# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import random

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

warnings.filterwarnings("ignore")

print("##################")
print("# IMPORTS LOADED #")
print("##################")
print("\n")

# Util_stat.py

In [ ]:
def haversine(pred, gt):
    """
    Havarsine distance between two points on the Earth surface.

    Parameters
    -----
    pred: numpy array of shape (N, 2)
        Contains predicted (LATITUDE, LONGITUDE).
    gt: numpy array of shape (N, 2)
        Contains ground-truth (LATITUDE, LONGITUDE).

    Returns
    ------
    numpy array of shape (N,)
        Contains haversine distance between predictions
        and ground truth.
    """
    pred_lat = np.radians(pred[:, 0])
    pred_long = np.radians(pred[:, 1])
    gt_lat = np.radians(gt[:, 0])
    gt_long = np.radians(gt[:, 1])

    dlat = gt_lat - pred_lat
    dlon = gt_long - pred_long

    a = np.sin(dlat/2)**2 + np.cos(pred_lat) * \
        np.cos(gt_lat) * np.sin(dlon/2)**2

    d = 2 * 6371 * np.arctan2(np.sqrt(a), np.sqrt(1-a))

    return d


def load_data(csv_path, nb_rows=None):
    """
    Reads a CSV file (train or test) and returns the data contained.

    Parameters
    ----------
    csv_path : String
        Path to the CSV file to be read.
        e.g., "train.csv"
    nb_rows : Integer, optional
        Number of rows to load from the dataset. If None, load the full dataset.

    Returns
    -------
    data : Pandas DataFrame 
        Data read from CSV file.
    n_samples : Integer
        Number of rows (samples) in the dataset.
    """
    data = pd.read_csv(csv_path, index_col="TRIP_ID")

    # Sample random rows if nb_rows is specified
    if nb_rows is not None:
        data = data.sample(n=nb_rows)

    return data, len(data)

def write_submission(trip_ids, destinations, file_name="submission"):
    """
    This function writes a submission csv file given the trip ids, 
    and the predicted destinations.

    Parameters
    ----------
    trip_id : List of Strings
        List of trip ids (e.g., "T1").
    destinations : NumPy Array of Shape (n_samples, 2) with float values
        Array of destinations (latitude and longitude) for each trip.
    file_name : String
        Name of the submission file to be saved.
        Default: "submission".
    """
    n_samples = len(trip_ids)
    assert destinations.shape == (n_samples, 2)

    submission = pd.DataFrame(
        data={
            'LATITUDE': destinations[:, 0],
            'LONGITUDE': destinations[:, 1],
        },
        columns=["LATITUDE", "LONGITUDE"],
        index=trip_ids,
    )

    # Write file
    submission.to_csv(file_name + ".csv", index_label="TRIP_ID")

# Datasets loading

### Train dataset loading

In [ ]:
# You can choose the limitation of the Train set, here 1000 rows
train_data, len_train_data = load_data("train.csv", nb_rows=1000)

print("#################")
print("# TRAIN LOADED #")
print("#################")
print("\n")

train_data

### Test dataset loading

In [ ]:
test_data, len_data_test = load_data("test.csv")
test_dataset_trips_ids = list(test_data.index)

print("################")
print("# TEST LOADED #")
print("################")
print("\n")

test_data

# Dataset analysis before preprocessing

### Dataset polyline lengths analysis

In [ ]:
def analyze_polyline_lengths(data):
    # Storing lengths of each polyline
    polyline_lengths = [len(eval(polyline)) for polyline in data['POLYLINE']]

    # Calculate Q1 (25th percentile) and Q3 (75th percentile)
    Q1 = np.percentile(polyline_lengths, 25)
    Q3 = np.percentile(polyline_lengths, 75)

    # Calculate Interquartile Range (IQR)
    IQR = Q3 - Q1

    # Calculate minimum and maximum non-outlier thresholds
    min_non_outlier = max(Q1 - 1.5 * IQR, 2)
    max_non_outlier = Q3 + 1.5 * IQR

    # Calculate mean and median of polyline lengths
    mean_length = np.mean(polyline_lengths)
    median_length = np.median(polyline_lengths)

    # Print results
    print("Non-outlier minimum polyline length:", min_non_outlier)
    print("Non-outlier maximum polyline length:", max_non_outlier)
    print("Q1:", Q1)
    print("Q3:", Q3)
    print("IQR:", IQR)
    print("Mean length of polyline:", mean_length)
    print("Median length of polyline:", median_length)

    # Creating boxplot of polyline lengths
    plt.figure(figsize=(10, 6))
    plt.boxplot(polyline_lengths)
    plt.title("Boxplot of Polyline Lengths")
    plt.ylabel("Length of Polyline")
    plt.show()

    return max_non_outlier

##### Train dataset polyline lengths analysis

In [ ]:
max_non_outlier_train_data = analyze_polyline_lengths(train_data)

print("#######################################")
print("# TRAIN POLYLINE LENGTH ANALYSIS DONE #")
print("#######################################")
print("\n")

##### Test dataset polyline lengths analysis

In [ ]:
max_non_outlier_test_data = analyze_polyline_lengths(test_data)

print("######################################")
print("# TEST POLYLINE LENGTH ANALYSIS DONE #")
print("######################################")
print("\n")

# Dataset preprocessing

In [ ]:
def preprocess_dataset(data, max_non_outlier):

    # Drop 'DAY_TYPE' column (because always of type A)
    data = data.drop('DAY_TYPE', axis=1)

    # Keep only rows where 'MISSING_DATA' value is False
    data = data[data['MISSING_DATA'] == False]

    # Drop 'MISSING_DATA' column
    data = data.drop('MISSING_DATA', axis=1)

    # Keep only rows where 'POLYLINE' is not an empty list
    data = data[data['POLYLINE'] != '[]']
    
    # Replace NaN values in 'ORIGIN_CALL' column with 0
    data['ORIGIN_CALL'] = data['ORIGIN_CALL'].fillna(0)

    # Replace NaN values in 'ORIGIN_STAND' column with 0
    data['ORIGIN_STAND'] = data['ORIGIN_STAND'].fillna(0)
    
    # Replace categorical values in CALL_TYPE with numerical values
    data['CALL_TYPE'] = data['CALL_TYPE'].replace(['A', 'B', 'C'], [0, 1, 2])

    for index, row in data.iterrows():

        # extract 'POLYLINE' value of current row
        polyline = eval(row['POLYLINE'])

        # Calculate duration of the ride
        duration = len(polyline) * 15
        # Create new column for ride duration
        data.at[index, 'DURATION'] = duration

        polyline_length = len(polyline)

        # Create new colum in data for selected longitude and latitude of the points selected

         # Assign first point to LONG_0 and LAT_0
        data.at[index, 'LONG_0'] = polyline[0][0]
        data.at[index, 'LAT_0'] = polyline[0][1]

        if polyline_length == 1 or polyline_length == 2:
            # Use first point for all if only two points
            data.at[index, 'LONG_1'] = polyline[0][0]
            data.at[index, 'LAT_1'] = polyline[0][1]
            data.at[index, 'LONG_2'] = polyline[0][0]
            data.at[index, 'LAT_2'] = polyline[0][1]

            # final longitude and latitude from polyline assigned to new END_LONG and END_LATcolumn
            data.at[index, 'END_LONG'] = polyline[-1][0]
            data.at[index, 'END_LAT'] = polyline[-1][1]

        elif polyline_length == 3:
            # Use second point for LONG_1, LAT_1, LONG_2, and LAT_2
            data.at[index, 'LONG_1'] = polyline[1][0]
            data.at[index, 'LAT_1'] = polyline[1][1]
            data.at[index, 'LONG_2'] = polyline[1][0]
            data.at[index, 'LAT_2'] = polyline[1][1]
            
            # final longitude and latitude from polyline assigned to new END_LONG and END_LATcolumn
            data.at[index, 'END_LONG'] = polyline[-1][0]
            data.at[index, 'END_LAT'] = polyline[-1][1]

        elif polyline_length == 4:
            # Assign second point for LONG_1 and LAT_1
            data.at[index, 'LONG_1'] = polyline[1][0]
            data.at[index, 'LAT_1'] = polyline[1][1]

            # Assign third point for LONG_2 and LAT_2
            data.at[index, 'LONG_2'] = polyline[2][0]
            data.at[index, 'LAT_2'] = polyline[2][1]

            # final longitude and latitude from polyline assigned to new END_LONG and END_LATcolumn
            data.at[index, 'END_LONG'] = polyline[-1][0]
            data.at[index, 'END_LAT'] = polyline[-1][1]

        else:
            midpoint_idx = (polyline_length // 2)

            # Assign midpoint to LONG_1 and LAT_1
            data.at[index, 'LONG_1'] = polyline[midpoint_idx][0]
            data.at[index, 'LAT_1'] = polyline[midpoint_idx][1]

            # Choose a random point for LONG_2 and LAT_2
            random_idx = random.randint(midpoint_idx + 1, polyline_length - 2)
            data.at[index, 'LONG_2'] = polyline[random_idx][0]
            data.at[index, 'LAT_2'] = polyline[random_idx][1]

            if polyline_length > max_non_outlier:
                # Choose random point for END_LONG and END_LAT
                end_idx = random.randint(random_idx + 1, polyline_length - 1)
                data.at[index, 'END_LONG'] = polyline[end_idx][0]
                data.at[index, 'END_LAT'] = polyline[end_idx][1]
            else:
                # Use the last point for END_LONG and END_LAT
                data.at[index, 'END_LONG'] = polyline[-1][0]
                data.at[index, 'END_LAT'] = polyline[-1][1]

    # Drop original 'POLYLINE' column
    data = data.drop('POLYLINE', axis=1)

    # Convert TIMESTAMP from Unix time to datetime
    data['TIMESTAMP'] = pd.to_datetime(data['TIMESTAMP'], unit='s')

    # Extract week, day, and quarter
    data['WEEK'] = data['TIMESTAMP'].dt.isocalendar().week
    data['DAY'] = data['TIMESTAMP'].dt.weekday
    data['QUARTER'] = (data['TIMESTAMP'].dt.hour * 4) + \
        (data['TIMESTAMP'].dt.minute // 15)

    # Ensure WEEK values are within 1-52
    data['WEEK'] = data['WEEK'].clip(1, 52)

    # Drop original 'TIMESTAMP' column
    data = data.drop('TIMESTAMP', axis=1)

    return data

##### Train dataset preprocessing

In [ ]:
preprocessed_train_data = preprocess_dataset(train_data, max_non_outlier_train_data)

print("######################")
print("# TRAIN PREPROCESSED #")
print("######################")
print("\n")

preprocessed_train_data

##### Test dataset preprocessing

In [ ]:
preprocessed_test_data = preprocess_dataset(test_data, max_non_outlier_test_data)

print("#####################")
print("# TEST PREPROCESSED #")
print("#####################")
print("\n")

preprocessed_test_data

# Dataset longitude & latitude analysis after preprocessing

In [ ]:
def plot_boxplots_for_specific_columns(data):
    # Define specific columns to plot
    specific_columns = [col for col in data.columns if 'LONG_' in col or 'LAT_' in col or col in ['END_LONG', 'END_LAT']]

    # Generate boxplots for each specific column
    for column in specific_columns:
        plt.figure(figsize=(6, 4))
        data[column].plot.box()
        plt.title(column)
        plt.show()

### Train dataset longitude and latitude analysis

In [ ]:
plot_boxplots_for_specific_columns(preprocessed_train_data)

print("################################")
print("# TRAIN LONG/LAT ANALYSIS DONE #")
print("################################")
print("\n")

### Test dataset longitude and latitude analysis

In [ ]:
plot_boxplots_for_specific_columns(preprocessed_test_data)

print("###############################")
print("# TEST LONG/LAT ANALYSIS DONE #")
print("###############################")
print("\n")

# Dataset splitting

In [ ]:
def split_dataset(data):
    # Separate X = all points except last one and y = last one
    X, y = data.drop(['END_LONG', 'END_LAT'], axis=1), data[['END_LONG', 'END_LAT']]
    return X, y

### Train dataset splitting

In [ ]:
X_train, y_train = split_dataset(preprocessed_train_data)

print("##################")
print("# TRAIN SPLITTED #")
print("##################")
print("\n")

In [ ]:
X_train

In [ ]:
y_train

### Test dataset splitting

In [ ]:
X_test, y_test = split_dataset(preprocessed_test_data)

print("#################")
print("# TEST SPLITTED #")
print("#################")
print("\n")

In [ ]:
X_test

In [ ]:
y_test

# Model hyperparameters tuning

Trying to find the best hyperparameters for kNN, Decision tree and Ridge using GridSearchCV. Linear regression doesn't need hyperparameters, and it's too time-consuming to do so for random forest, so manual tuning is required. Currently we could try: (n_estimators, max_depth, min_samples_split, min_samples_leaf) = (1000, 1000, 10, 4)

In [ ]:
#models_to_tune = [KNeighborsRegressor(), DecisionTreeRegressor(), Ridge()]
models_to_tune = [DecisionTreeRegressor(), Ridge()]

regression_params = [
                        # KNN
                        #{
                            #'n_neighbors': [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024],
                            #'weights': ['uniform', 'distance'],
                            #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                        #},
                        # DT
                        {
                            'max_depth': [10, 100, 1000], 
                            'min_samples_split': [2, 5, 10],
                            'min_samples_leaf': [1, 2, 4],
                        },
                        # Ridge
                        {
                            'alpha': [0.1, 1, 10, 100, 1000],
                        }
                    ]

print("GridSearchCV:\n")
tuned_models = []
tuned_models.append(KNeighborsRegressor(n_neighbors=256, weights='distance', algorithm='auto'))
for idx, model in enumerate(models_to_tune):
    grid = GridSearchCV(model, param_grid=regression_params[idx], cv=5, verbose=1)
    grid.fit(X_train, y_train)
    tuned_model = grid.best_estimator_
    tuned_models.append(tuned_model)
    print(f"Best parameters for {type(model).__name__}: {grid.best_params_}")
    print("\n")

# add LR and RF manually tuned models
tuned_models.append(LinearRegression())
tuned_models.append(RandomForestRegressor(n_estimators=1000, max_depth=1000, min_samples_split=10, min_samples_leaf=4, random_state=0))
    
print("###############################")
print("# HYPERPARAMETERS TUNING DONE #")
print("###############################")
print("\n")

# Model fitting & prediction

In [ ]:
def fit_predict(X_train, y_train, X_test, model):
    """
    Generic function for fitting the model and predicting the destination

    Args:
        X_train (pandas.core.frame.DataFrame'): data frame of train input
        y_train (pandas.core.frame.DataFrame'): data frame of train output
        X_test (pandas.core.frame.DataFrame'): data frame of test input
        model (_type_): model like DecisionTreeRegressor(max_depth=param, random_state=(RAND_STATE*rand_value))

    Returns:
        numpy.ndarray': destinations predicted of shape (nb_of_dest, 2)
    """
    
    model.fit(X_train, y_train)
    
    print("Fitting done")
    
    prediction = model.predict(X_test)
    
    print("Prediction done")
    
    # Swap columns (longitude, latitude) -> (latitude, longitude)
    prediction = np.stack((prediction[:, 1], prediction[:, 0]), axis=-1)

    return prediction

### Make predictions

In [ ]:
predictions = []

# Fit and predict
for idx, model in enumerate(tuned_models):
    print("Starting model nb", idx)
    curr_model_pred = fit_predict(X_train, y_train, X_test, model)
    predictions.append(curr_model_pred)
    print("\n")
    
print("####################")
print("# PREDICTIONS DONE #")
print("####################")
print("\n")

# Make csv submission

In [ ]:
# Write submission for each prediction
for idx, prediction in enumerate(predictions):
    write_submission(test_dataset_trips_ids, prediction, f"submission_{idx}")
    
print("############")
print("# CSV DONE #")
print("############")
print("\n")

# Evaluate predictions precision

In [ ]:
# Load ground truth data
ground_truth = pd.read_csv('solutions.csv', index_col='TRIP_ID')

# Metrics calculation for each submission file
for idx in range(len(predictions)):
    submission = pd.read_csv(f"submission_{idx}.csv", index_col='TRIP_ID')
    submission = submission.loc[ground_truth.index]

    gt_public = ground_truth[ground_truth['PUBLIC'] == True]
    gt_private = ground_truth[ground_truth['PUBLIC'] == False]

    sub_public = submission.loc[gt_public.index]
    sub_private = submission.loc[gt_private.index]

    # Haversine
    hvpublic = haversine(gt_public[['LATITUDE','LONGITUDE']].values, sub_public[['LATITUDE','LONGITUDE']].values)
    hvprivate = haversine(gt_private[['LATITUDE','LONGITUDE']].values, sub_private[['LATITUDE','LONGITUDE']].values)

    # Mean, Std, MSE, MAE
    mean_public = hvpublic.mean()
    std_public = hvpublic.std()
    mse_public = mean_squared_error(gt_public[['LATITUDE','LONGITUDE']].values, sub_public[['LATITUDE','LONGITUDE']].values)
    mae_public = mean_absolute_error(gt_public[['LATITUDE','LONGITUDE']].values, sub_public[['LATITUDE','LONGITUDE']].values)

    mean_private = hvprivate.mean()
    std_private = hvprivate.std()
    mse_private = mean_squared_error(gt_private[['LATITUDE','LONGITUDE']].values, sub_private[['LATITUDE','LONGITUDE']].values)
    mae_private = mean_absolute_error(gt_private[['LATITUDE','LONGITUDE']].values, sub_private[['LATITUDE','LONGITUDE']].values)

    # Printing results
    print(f"Metrics for submission {idx}:")
    print(f"Public - Mean: {mean_public}, Std: {std_public}, MSE: {mse_public}, MAE: {mae_public}")
    print(f"Private - Mean: {mean_private}, Std: {std_private}, MSE: {mse_private}, MAE: {mae_private}\n")
    
print("###################################")
print("# PREDICTIONS PRECISION EVAL DONE #")
print("###################################")
print("\n")